In [ ]:
import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark import SparkContext
conf = SparkConf()
conf.setMaster('yarn')
conf.setAppName('spark-nltk')
spark_session = SparkContext(conf=conf)

In [9]:
import os
import pyspark
from pyspark.sql import SparkSession
os.environ['PYSPARK_PYTHON'] = "./environment/bin/python"
spark_session = SparkSession.builder.master("yarn") \
                    .appName('spark-yarn-conda_env') \
                    .config(
                        "spark.archives",  # 'spark.yarn.dist.archives' in YARN.
                        "hdfs://192.168.2.250:9000/user/ubuntu/share/envs/pyspark_conda_env.tar.gz#environment") \
                    .getOrCreate()

In [1]:
import os
from pyspark.sql import SparkSession

os.environ['PYSPARK_PYTHON'] = "./environment/bin/python"
spark = SparkSession.builder.config(
    "spark.archives",  # 'spark.yarn.dist.archives' in YARN.
    "hdfs://192.168.2.250:9000/user/ubuntu/share/envs/pyspark_conda_env.tar.gz#environment").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
spark_session = spark

In [10]:
# Load comments into a Data Frame
commentsDF = spark_session.read.json("hdfs://192.168.2.250:9000/user/ubuntu/1000lines_RC_2009-09")

In [ ]:
commentsDF.printSchema()

In [11]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import operator
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [12]:
# Creates a temporary view using the DataFrame
commentsDF.createOrReplaceTempView("body_score")

# SQL statements can be run by using the sql methods provided by spark
body_scoreDF = spark_session.sql("SELECT body, score FROM body_score ORDER BY score DESC LIMIT 20")

In [ ]:
'''
import flair
from flair.models import TextClassifier
from flair.data import Sentence
sia = TextClassifier.load('en-sentiment')
'''

In [13]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType

sia = SentimentIntensityAnalyzer()

def fl_sentiment(text):
    sentence = Sentence(text)
    sia.predict(sentence)
    return(str(sentence.labels[0]))

def fl_sentiment2(text):
    return(sia.polarity_scores(text))

fl_sentimentUDF = udf(fl_sentiment2, StringType())

In [14]:
body_scoreDF.withColumn("sentiment_score",fl_sentimentUDF(col("body"))).select("body", "score", "sentiment_score").show()

+--------------------+-----+--------------------+
|                body|score|     sentiment_score|
+--------------------+-----+--------------------+
|My friend linked ...|  680|{neg=0.043, pos=0...|
|This was a conver...|  550|{neg=0.108, pos=0...|
|Computes perfectl...|  419|{neg=0.0, pos=0.5...|
|There a few good ...|  370|{neg=0.197, pos=0...|
|           [deleted]|  367|{neg=0.0, pos=0.0...|
|&gt;My cousin is ...|  338|{neg=0.182, pos=0...|
|My friend once ac...|  301|{neg=0.0, pos=0.1...|
|**New Jersey Douc...|  300|{neg=0.0, pos=0.3...|
|my wife's brother...|  281|{neg=0.06, pos=0....|
|Playing ultimate ...|  248|{neg=0.178, pos=0...|
|A guy I know to o...|  238|{neg=0.077, pos=0...|
|Yeah, but you don...|  233|{neg=0.0, pos=0.3...|
|The two girls who...|  206|{neg=0.084, pos=0...|
|This was a great ...|  152|{neg=0.108, pos=0...|
|A long time ago, ...|  143|{neg=0.098, pos=0...|
|My girlfriend at ...|  118|{neg=0.139, pos=0...|
|&gt;Random Custom...|  101|{neg=0.0, pos=0.0...|


In [15]:
spark_session.stop()

In [20]:
# pip install vaderSentiment
# pip install nltk

In [22]:
def analyze_sentiment(text):
    """Determines the sentiment of a given text. 
    Returns -1 if text sentiment is neither positive nor negative.
    Returns 1 if text sentiment is mainly positive
    Returns 0 if text sentiment is mainly negative. """
    sia = SentimentIntensityAnalyzer()
    scores = sia.polarity_scores(text)
    pos = scores['pos']
    neg = scores['neg']
    if pos == neg:
        return -1
    elif pos > neg:
        return 1
    else:
        return 0

In [26]:
analyze_sentiment('I am sitting here')

0

In [ ]:
lines.take(2)

In [2]:
spark_context.stop()

NameError: name 'spark_context' is not defined